<center><h1>Making Merger Tree from m3p output</h1></center>

In [ ]:
import numpy as np
import m3p_merger

In [ ]:
# Peak patch data
pathPrefix = "/usr/users/ellis/PeakPatch/m3p/axOut/"

# Unevolved Density fields
DensPath = "/usr/users/ellis/bin/InputFiles/512"
DensSuffix = ['/L6N3_07/axion.m.00115_L6N3_07','/L6N3_06/axion.m.00115_L6N3_06','/L6N3_01/axion.m.00115_L6N3_01']

In [ ]:
ppFile = pathPrefix + "../inputs/inputs.axsmall"
densFile = DensPath+DensSuffix[0]

peak_list, boxsize = m3p_merger.MakePeakList(ppFile)

<h2><center>Find which trees are interesting</center></h2>

In [ ]:
for i in range(452):
    out = m3p_merger.BuildMergerTree(peak_list, ppFile, i)
    if out[4].size>0:
        print(i, len(out[4]))

<h2><center>Calculate Merger Tree</center></h2>

In [ ]:
out = m3p_merger.BuildMergerTree(peak_list, ppFile, 335)

<h2><center>Plot all peaks on each other</center></h2>

In [ ]:
m3p_merger.plotMergerPatches(out, ppFile)

<h2><center>Plot merger tree</center></h2>

In [ ]:
peak_list, boxsize = m3p_merger.MakePeakList(ppFile)
m3p_merger.plotMergerTree(out, ppFile)

In [ ]:
out